# DidgeLab tutorial part 4 - Advanced Concepts

## Running evolution from the command line

We used the jupyter notebooks, but I did not use jupyter notebook with DidgeLab until I wrote this tutorial. Usually I use command line tools. Here I want to show how I run an evolution over the command line. Usually I do this on a strong server computer.

My evolutionary experiments are stored in folder cad/evo. In order to run e.g. the MbeyaEvo, I open a terminal in the src folder and then type

```
python -m cad.evo.evolve_mbeya -n_threads 50 -n_generations 1000 -n_generation_size 100 -n_poolsize 10
```

Parameters `n_generations` and `n_generation_size` should be familiar from the last tutorial. `n_poolsize` is the size of the mutant pool. `n_threads` defines how many threads compute the loss function in parallel. A generation size of 100 means that DidgeLab creates 100 mutations in each pool, which means 1000 mutations. Creating a mutation is very fast. But computing the acoustical simulation takes a while. So with `n_threads=50`, the system will start 50 threads that compute losses and the acoustical simulation in parallel. When the loss for all 1000 mutations in the generation is computed, DidgeLab selects the best mutant in each pool (or the father, if no mutant was better) and starts the next generation. It does not make sense to have a number of `n_threads` higher than the number of cpu cores in your computer.

## A more elaborate loss function.

Lets take a look at the source code of `evolve_mbeya.py`. 

The loss function has several objectives:

* fundamental loss: the fundamental note should be D1 (-31)
* octave_loss: the first toot should be an octave
* tuning_loss: all other toots should be tuned in the minor scale. The scale parameter has value `[0,2,3,5,7,9,10]`. So all "allowed" toots are fundamental + 0 semitones (which is the D), fundamental + 2 semitones (E), fundamental + 3 semitones (F) and so on, so all toots should have notes D, E, F, G, A, Bb, C, in whichever octave.
* volume loss: Each resonant frequency should be as loud as possible
* n_note_loss: The didge should have at least 4 toots. Otherwise, the natural evolution will find didgeridoos with as little toots as possible, to keep tuning_loss and octave_loss low. Also, many toots are cool.
* diameter_loss: A didgeridoo should always get wider along the bore. If it gets narrower then it will get more quiet. Therefore, we want to penalize forms that go narrower.

Also you can see that the loss function is weighted, to make some parts more or less important for the final loss. The loss function and the weights itself are the result of 

In [1]:
# do not run this code cell. It will not work from the jupyter notebook and if you tweak it to run 
# then it will run hours and hours and hours... 
# Therefore i put this sys.exit(0) in the first line to stop it if you accidently run it.
import sys
sys.exit(0)

from cad.calc.pipeline import Pipeline, ExplorePipelineStep, OptimizeGeoStep, PipelineStartStep, FinetuningPipelineStep, AddPointOptimizerExplore, AddPointOptimizerFinetune
from cad.common.app import App
from cad.calc.mutation import ExploringMutator, FinetuningMutator, MutantPool
from cad.calc.parameters import MbeyaShape
from cad.calc.loss import LossFunction, TootTuningHelper, diameter_loss, single_note_loss
import numpy as np
from cad.calc.geo import geotools
from cad.cadsd.cadsd import CADSD, cadsd_octave_tonal_balance
from cad.calc.conv import note_to_freq, note_name, freq_to_note_and_cent
import math
import numpy as np
from cad.calc.geo import Geo
from cad.ui.evolution_ui import EvolutionUI
from cad.calc.util.losslog import LossLog
from cad.calc.util.cad_logger import LossCADLogger
import logging

class MbeyaLoss(LossFunction):

    # fundamental: note number of the fundamental
    # add_octave: the first toot is one octave above the fundamental
    # scale: define the scale of the toots of the didgeridoo as semitones relative from the fundamental
    # target_peaks: define the target peaks as list of math.log(frequency, 2). overrides scale 
    # n_notes: set > 0 to determine the number of impedance peaks (above fundamental and add_octave)
    # weights: override the default weights
    # {
    #     "tuning_loss": 8,
    #     "volume_loss": 0.5,
    #     "octave_loss": 4,
    #     "n_note_loss": 5,
    #     "diameter_loss": 0.1,
    #     "fundamental_loss": 8,
    # }
    def __init__(self, fundamental=-31, add_octave=True, n_notes=-1, scale=[0,2,3,5,7,9,10], target_peaks=None, weights={}):
        LossFunction.__init__(self)

        self.weights={
            "tuning_loss": 8,
            "volume_loss": 0.5,
            "octave_loss": 4,
            "n_note_loss": 5,
            "diameter_loss": 0.1,
            "fundamental_loss": 8,
        }
        for key, value in weights.items():
            if key not in self.weights:
                raise Exception(f"Unknown weight {key}")
            self.weights[key]=value


        self.scale=scale
        self.fundamental=fundamental
        self.add_octave=add_octave
        self.n_notes=n_notes

        if target_peaks is not None:
            self.target_peaks=target_peaks
        else:
            self.scale_note_numbers=[]
            for i in range(len(self.scale)):
                self.scale_note_numbers.append(self.scale[i]+self.fundamental)

            n_octaves=10
            self.target_peaks=[]
            for note_number in self.scale_note_numbers:
                for i in range(0, n_octaves):
                    transposed_note=note_number+12*i
                    freq=note_to_freq(transposed_note)
                    freq=math.log(freq, 2)
                    self.target_peaks.append(freq)

    def get_loss(self, geo, context=None):

        fundamental=single_note_loss(-31, geo)*self.weights["fundamental_loss"]
        octave=single_note_loss(-19, geo, i_note=1)*self.weights["octave_loss"]

        notes=geo.get_cadsd().get_notes()
        tuning_loss=0
        volume_loss=0

        start_index=1
        if self.add_octave:
            start_index+=1
        if len(notes)>start_index:
            for ix, note in notes[start_index:].iterrows():
                f1=math.log(note["freq"],2)
                closest_target_index=np.argmin([abs(x-f1) for x in self.target_peaks])
                f2=self.target_peaks[closest_target_index]
                tuning_loss += math.sqrt(abs(f1-f2))
                volume_loss += math.sqrt(1/(note["impedance"]/1e6))

        tuning_loss*=self.weights["tuning_loss"]
        volume_loss*=self.weights["volume_loss"]
        
        n_notes=self.n_notes+1
        if self.add_octave:
            n_notes+=1
        n_note_loss=max(n_notes-len(notes), 0)*self.weights["n_note_loss"]

        d_loss = diameter_loss(geo)*self.weights["diameter_loss"]

        loss={
            "tuning_loss": tuning_loss,
            "volume_loss": volume_loss,
            "n_note_loss": n_note_loss,
            "diameter_loss": d_loss,
            "fundamental_loss": fundamental,
            "octave_loss": octave,
        }
        loss["loss"]=sum(loss.values())
        return loss

if __name__=="__main__":
    try:
        App.full_init("evolve_penta")

        losslogger=LossCADLogger()

        loss=MbeyaLoss(n_notes=3)    
        father=MbeyaShape()
        initial_pool=MutantPool.create_from_father(father, App.get_config()["n_poolsize"], loss)

        pipeline=Pipeline()

        pipeline.add_step(ExplorePipelineStep(ExploringMutator(), loss, initial_pool, n_generations=200, generation_size=70))
        pipeline.add_step(FinetuningPipelineStep(FinetuningMutator(), loss, n_generations=500, generation_size=30))

        for i in range(10):
            pipeline.add_step(AddPointOptimizerExplore(loss, n_generations=100, generation_size=30))
            pipeline.add_step(AddPointOptimizerFinetune(loss, n_generations=100, generation_size=30))

        ui=EvolutionUI()

        pipeline.run()

    except Exception as e:
        App.log_exception(e)


SystemExit: 0

/home/jan/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## EvolutionUI

The line EvolutionUI() starts the user interface to monitor the evolution. 

It shows the first mutant in the pool. Pressing x and y switches to the currently best mutant in each pool. It shows these information:

* At first it shows a table with various information about the didgeridoo. 
* Then it shows the resonant frequencies
* Then it shows an "FFT" diagram. It shows the sound volume for the different frequencies from 1-999 Hz.

![alt text](evolution_ui.png)


## The output folder

DidgeLab generates an output folder. It is located in the folder output/ and generates a folder named after the date and time when it was started. The last folder should always correspond to the last run of DidgeLab. It contains these files:

* cadlogger.log - a logfile for the loss. We explain this shortly when we show the Evolution Report.
* log.txt - a log file. This is mainly used for debugging the code and is not explained in this tutorial.
* a folder results/. This folder contains the many .pkl files which are named 0.pkl, 1.pkl, 2.pkl, ... Each file is the mutant pool at the end of each pipeline step. So 0.pkl is the mutant pool after the first step, 1.pkl after the 2nd, and so on.

## Evolution Report

The evolution report creates a PDF file with many information about the evolution. You can generate it using

```
python -m bin.make_didge_report -infile INFILE
```

Instead of INFILE you write the path, either to the output folder or to a specific .pkl file from the results folder. Adding the output folder will read the latest .pkl file. There is an example of a didge report in this folder, in the file `report.pdf`. Lets walk you through that report:

![alt text](report1.png)


* It generates one section for each didgeridoo in the mutant pool. Here we show section 1 only, so the first mutant in the pool.
* Section 1.1 shows the shape and some information about the loss.

![alt text](report2.png)


The next section shows the tuning of the toots / the resonant frequencies.

![alt text](report3.png)
![alt text](report4.png)


We omit 1.3 because it is not useful in this example. It shows the evolution parameters. Section 1.4 shows the sound spektra: The impedance chart and the sound spektra of ground tone and 1st overblow tone.

## CADLogger

If you input the whole folder instead a specific .pkl file and in case you initialized a LossCADLogger() as we did in above code example, the report contains a section Loss Report. This loss report here is taken from another example because I no longer had a loss report from above MbeyaExample available.

It shows the best loss over all mutants in the mutant pool over the "accumulated generation". So in case we have 3 pipeline steps with 500 generations each, then the accumulated generation ranges from 1-1500. 1-500 is the first pipeline step, 501-1000 the 2nd and so on. The vertical blue lines show that the next pipeline step started here.

It can give many information to the advanced user. From this report e.g. we can see that the 2nd pipeline step improved the total loss (blue) only in the beginning. So the 2nd pipeline step was maybe too long. The first pipeline step had a good length, because it improved most of the time and then reached a plateau. Also in the 2nd step, the tuning improved but, at the same time, the diameter loss got higher. 

It is useful to find out why the evolution creates a certain didgeridoo shape. Sometimes e.g., one part of the loss is simply too small and therefore does not influence the evolution much.

![alt text](cadlogger.png)


## Additional tools

The code contains additional tools which I summarize briefly here:

* `src/blender`
Contains codes to export a geometry to [Blender](https://www.blender.org/). The idea is to use blender to create models for 3d printing.
* `src/bin/didge_console_report.py` Create a report about a Didgeridoo geometry on the console
* `src/bin/geo_2_didgmo.py` Convert a DidgeLab geometry to a Didgmo geometry
* `src/bin/explorer.py` Explore a mutant pool. This tool was replaced by make_didge_report.py
* `src/bin/geo2spektrum.py` Generate various reports and visualizations about a Didgeridoo